In [1]:
import spacy
from nltk.corpus import wordnet
import nltk

nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

In [2]:
def get_synonym(word):
    synonyms = []
    for syn in wordnet.synsets(word, pos=wordnet.ADJ):
        for lemma in syn.lemmas():
            # Αποφεύγουμε το ίδιο το αρχικό λέξη
            if lemma.name().lower() != word.lower():
                synonyms.append(lemma.name().replace('_', ' '))
    if synonyms:
        return synonyms[0]  # Παίρνουμε το πρώτο συνώνυμο
    else:
        return word  # Αν δεν βρεθεί συνώνυμο, επιστρέφουμε την ίδια λέξη

In [3]:
def replace_adjectives(sentence):
    doc = nlp(sentence)
    new_tokens = []
    for token in doc:
        if token.pos_ == "ADJ":
            new_word = get_synonym(token.text)
            new_tokens.append(new_word)
        else:
            new_tokens.append(token.text)
    return " ".join(new_tokens)

In [4]:
sentence1 = "Overall, let us make sure all are safe and celebrate the outcome with strong coffee and future targets" 
sentence2 = "Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives"
new_sentence1 = replace_adjectives(sentence1)
new_sentence2 = replace_adjectives(sentence2)
print("Original:", sentence1, sentence2)
print("Modified:", new_sentence1, new_sentence2)

Original: Overall, let us make sure all are safe and celebrate the outcome with strong coffee and future targets
Modified: Overall , let us make certain all are dependable and celebrate the outcome with potent coffee and next targets


ΕΡΩΤΗΜΑ 2
1ος τροπος 

In [6]:
import torch
from transformers import pipeline

from sentence_splitter import SentenceSplitter
splitter = SentenceSplitter(language='en')

In [7]:
paraphrase_Pegasus_pipeline = pipeline("text2text-generation", model="tuner007/pegasus_paraphrase", device=0 if torch.cuda.is_available() else -1)

text1 = "Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Hope you too, to enjoy it as my deepest wishes. Thank your message to show our words to the doctor, as his next contract checking, to all of us. I got this message to see the approved message. In fact, I have received the message from the professor, to show me, this, a couple of days ago. I am very appreciated the full support of the professor, for our Springer proceedings publication."

text2 = "“During our final discuss, I told him about the new submission — the one we were waiting since last autumn, but the updates was confusing as it not included the full feedback from reviewer or maybe editor? Anyway, I believe the team, although bit delay and less communication at recent days, they really tried best for paper and cooperation. We should be grateful, I mean all of us, for the acceptance and efforts until the Springer link came finally last week, I think. Also, kindly remind me please, if the doctor still plan for the acknowledgments section edit before he sending again. Because I didn’t see that part final yet, or maybe I missed, I apologize if so. Overall, let us make sure all are safe and celebrate the outcome with strong coffee and future targets"

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


'Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Hope you too, to enjoy it as my deepest wishes. Thank your message to show our words to the doctor, as his next contract checking, to all of us. I got this message to see the approved message. In fact, I have received the message from the professor, to show me, this, a couple of days ago. I am very appreciated the full support of the professor, for our Springer proceedings publication.'

In [9]:
splitter = SentenceSplitter(language='en')
sentence_list1 = splitter.split(text1)
sentence_list2 = splitter.split(text2)

paraphrased_sentences1 = paraphrase_Pegasus_pipeline(sentence_list1, max_length=60, num_return_sequences=1)
paraphrased_sentences2 = paraphrase_Pegasus_pipeline(sentence_list2, max_length=60, num_return_sequences=1)
paraphrased_text1 = " ".join([result['generated_text'] for result in paraphrased_sentences1])
paraphrased_text2 = " ".join([result['generated_text'] for result in paraphrased_sentences2])

print(paraphrased_text1)
print(paraphrased_text2)


Our Chinese culture has a dragon boat festival that is celebrated today. Hope you enjoy it as much as I did. Your message was appreciated by the doctor, as his next contract check, to all of us. I saw the approved message after getting this message. I received the message from the professor a couple of days ago. The professor supported the Springer proceedings publication.
I told him about the new submission we were waiting for, but the updates were confusing as they didn't include the full feedback from the reviewer or editor. I think the team tried their best for paper and cooperation despite the recent delay and less communication. We should be thankful for the acceptance and efforts until the Springer link came last week, I think. If the doctor still plans for the acknowledgments section to be edited before he sends again, please remind me. I apologize if I missed that part final. Let's make sure all are safe and celebrate the outcome with coffee and targets.
